# Texas school accountability data 2016
This notebook has the scripts needed to cut, filter and analyze 2016 school accountability data from the Texas Education Association.

## Download the data
The 2016 data is already in the `data` folder inside this repo, but here's how you'd get it.

First, go to the [accountability data portal](https://rptsvr1.tea.texas.gov/perfreport/account/2015/) and click the "Download data" link on the left rail.

<img src="screenshots/1-portal-page.gif" style="border: 1px solid #ccc; margin: 20px auto 40px auto;" />

On the resulting page, click the "Campus-level Data" radio button, then scroll down and click "Continue."

<img src="screenshots/2-data-page.gif" style="border: 1px solid #ccc; margin: 20px auto 40px auto;" />

Finally, on the data download page, select "Tab delimited" from the select menu. Click the "Select all" button. Then click the "Download" button.

<img src="screenshots/3-download-page.gif" style="border: 1px solid #ccc; margin: 20px auto 40px auto;" />

I renamed the data file `2016-tx-school-acc-data.dat` and dropped it in the `data` folder.